In [ ]:
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [ ]:
# Load dataset from directory
data_dir = "./Dataset"  # Root folder containing subfolders of each class
dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir, image_size=(256, 256), batch_size=32, label_mode="int"  # Use "int" if labels are integers
)

class_names = dataset.class_names

In [ ]:
train_size = 0.8
train_ds = dataset.take(int(len(dataset) * train_size))
test_ds = dataset.skip(int(len(dataset) * train_size))

In [ ]:
train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(64, activation='relu'),
    Dense(18, activation='softmax')  # 18 output units for 18 classes
])

In [ ]:
optim = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    loss='sparse_categorical_crossentropy', optimizer=optim, metrics=['accuracy']
)


In [ ]:
model.fit(train_ds, epochs=10)

In [ ]:
model.evaluate(test_ds)

In [ ]:
for images, labels in test_ds.take(1):  # Takes the first batch
    batch_images = images.numpy()
    batch_labels = labels.numpy()

# Select a random index from the current batch
idx = random.randint(0, len(batch_images) - 1)

# Get the random image and its true label
img_to_predict = batch_images[idx]
true_label = batch_labels[idx]

# Display the selected image
plt.imshow(img_to_predict if img_to_predict.max() <= 1 else img_to_predict / 255.0)
plt.axis('off')
plt.title(f"True Label: {class_names[true_label]}")
plt.show()

# Get model prediction
y_pred = model.predict(img_to_predict.reshape(1, 256, 256, 3))

# Get predicted class name
predicted_class_index = np.argmax(y_pred)
predicted_class_name = class_names[predicted_class_index]


print(f"Predicted class: {predicted_class_name}")